# Projeto 1 - Ciência dos Dados

Nome: Arthur Pansini

Nome: Caio Ribeiro

## Contextualização
___

O tema escolhido para a realização desse projeto foi o super herói Homem Aranha, personagem que possui uma franquia de jogos vinculados a empresa Sony, além de uma série de obras cinematográficas produzidas pela Marvel Studios.

O objetivo que almejamos com o projeto é de analisar o que os consumidores ou possíveis consumidores dos conteúdos relacionados ao Homem Aranha comentam sobre estes na rede social Twitter. Para classificar a relevância dos Twittes observamos os comentários que tinham relação a alguma opinião ou sentimento a respeito das obras citadas acima, sejam eles bons ou ruins, todo o resto que destoava muito desses conceitos foi descartado. 


Exemplos de assuntos considerados relevantes:
- Críticas ao jogo do Homem Aranha produzido pela Sony.
- Comentários de quem foi o melhor ator a interpretar o Homem aranha nos cinemas.
- Elogios a alguma cena do filme do personagem.

<center><img src="aranha.gif" width=700 style="float: center; margin: 0px 0px 10px 10px"></center>

## Carregando bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os 

In [2]:
import emoji
import nltk
import re
from emoji import UNICODE_EMOJI

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\caior\OneDrive\Faculdade\2° PERÍODO\Ciência dos Dados\Projetos\Homem-Aranha


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
filename = 'Homem-Aranha.xlsx'

In [5]:
train = pd.read_excel(filename)
train.head()

Treinamento  Classificação
0  tá, com "god of war: ragnarok", "homem-aranha ...              0
1  @flamenguismo_ @venecasagrande os cara se enco...              0
2  assisti venom e amei. agora vou ver todos os f...              1
3  @gabriel71551446 @oswaldooell @n3m_ai @xboxnat...              1
4  sério mesmo, eu quero muito assistir homem ara...              1

In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head()

Teste  Classificação
0                                   amo homem aranha              1
1  @drika_nozes na verdade esse é o plot de homem...              0
2  @vicgabil @cinepop falou que vai estrear na me...              0
3  @goodnerd23 os dois👍 só n sei como vou aguenta...              0
4                @cacocardassi homem aranha, óbvio!!              1

___
## Classificador automático de sentimento


Foi considerado como relevante todo comentário referente ao universo Homem-Aranha (quadrinhos, filmes e jogos) que expressava opiniões ou sentimentos, sejam eles bons ou ruins, referente às obras, de forma que estes apresentassem algum tipo de embasamento, não sendo considerado como relevante, portanto, comentários desconexos ou pouco pertinentes com a análise crítica.

Passamos pelos seguintes passos de limpeza e manipulação dos comentários:
* Limpar as pontuações e caracteres como "@" e '!';
* Retirar os paragrafos existentes em alguns comentários;
* Dividir as palavras em listas para análise probalística de cada palavra existentes nos comentários.

###  Funções iniciais

In [7]:
# Retirando pontuações dos tweets
def cleanup(tweet):
    punctuation = '[(\n”\-/!.:?;,''"#)]'             
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern,'', tweet)
    return text_subbed

In [8]:
nltk.download('stopwords')
prep = nltk.corpus.stopwords.words('portuguese')
prep.append('pra')

def limpa_preposicao(lista):
    sem_prep=[]     
    for palavra in lista: 
        if not palavra in prep: 
            sem_prep.append(palavra)
    return sem_prep

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\caior\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Limpa @ e links dos elementos de uma lista
def limpa_marcacao(lista):
    sem_marc = []
    for palavra in limpa_preposicao(lista):
        if not 'https' in palavra:
            if not '@' in palavra:
                sem_marc.append(palavra)
        else:
            continue
    return sem_marc

In [10]:
# Limpa @ e links de uma string
def limpa_marcacao1(linha):
    sem_marc = []
    for palavra in limpa_preposicao(linha.split()):
        if not 'https' in palavra:
            if not '@' in palavra:
                sem_marc.append(palavra)
        else:
            continue
    return sem_marc

In [11]:
# Separa emojis seguidos, e os transcreve
def separa_emoji(tweet):
    modified=' '.join(emoji.get_emoji_regexp().split(tweet))
    modified=modified.split()
    for i,emoji1 in enumerate(modified):
        if emoji1 in UNICODE_EMOJI['pt']:
            modified[i]=UNICODE_EMOJI['pt'][emoji1].replace(':','')
        elif emoji1 in UNICODE_EMOJI['en']:
            modified[i]=UNICODE_EMOJI['en'][emoji1].replace(':','')
        else:
            continue
    modified=' '.join(modified)  
    return modified

In [12]:
# Aplica as funções anteriores e retorna uma lista de palavras do dataframe.Treinamento
def treinamento(dataframe):
    palavras = []
    for linha in dataframe.Treinamento:
        linha = separa_emoji(cleanup(linha))
        for palavra in limpa_marcacao(linha.split()):
            palavras.append(palavra)
    return palavras

In [13]:
# Aplica as funções anteriores e retorna uma lista de tweets (frases) do dataframe.Treinamento
def treinamento1(dataframe):
    frases = []
    for linha in dataframe.Treinamento:
        linha = limpa_marcacao1(separa_emoji(cleanup(linha)))
        frases.append(linha)
    return frases

___
## Um pouco sobre  o Classificador Naive-Bayes

Pretendemos construir um algoritimo que consiga tomar a decisão de agrupar uma frase como um assunto relevante ou não relevante sobre o tema ou produto Homem-Aranha. Utilizaremos o Naive Bayes para classificar textos baseado na frequência das palavras utilizadas, sendo assim conhecido por ser um classificador probabilístico que utiliza do Teorema de Bayes, visto de maneira mais aprofundada nas aulas de Ciência dos Dados.

###  Probabilidades Iniciais:

Primeiramente iremos calcular a probabilidade de palavras relevantes e irrelevantes aparecerem em nosso conjunto total:

$$ P_{relevante}= \frac{N° Palavras_{relevantes}}{N° Total_{palavras}} $$

$$ P_{irrelevante}= \frac{N° Palavras_{irrelevantes}}{N° Total_{palavras}} $$

De acordo com a teoria complementar de conjuntos:

$$ P_{relevante}+ P_{irrelevante}=1 $$

### Demais Probabilidades:

$P(R) \rightarrow $ Probabilidade de um Tweet ser relevante.

$P(I)  \ ou \ P(R^c)\rightarrow$ Probabilidade de um Tweet ser irrelevante.

$P(tweet) \rightarrow$ Probabilidade de determinado Tweet ocorrer.

As probabilidades que o nosso classificador precisa encontrar são: "Dado um tweet, qual a probabilidade dele ser relevante?" e "Dado um tweet, qual a probabilidade dele ser irrelevante?", sendo ambas, respectivamente, representadas por $P(R|tweet)$ e $P(I|tweet)$.

Dessa maneira, temos que:

$$ P(R|tweet)= \frac{P(tweet \cap R)}{P(tweet)} $$

$$ P(I|tweet)= \frac{P(tweet \cap I)}{P(tweet)} $$

Sabendo-se que $P(tweet \cap R)$ e $P(tweet \cap I)$ também podem ser encontrada pelas seguintes relações:

$$ P(tweet|R)= \frac{P(tweet \cap R)}{P(R)} $$

$$ P(tweet|I)= \frac{P(tweet \cap I)}{P(I)} $$

Obtemos que a probabilidade utilizada por Naive Bayes para classificar um texto como relevante ou não relevante são:

$$ P(R|tweet)= \frac{P(tweet|R) P(R)}{P(tweet)} $$

$$ P(I|tweet)= \frac{P(tweet|I) P(I)}{P(tweet)} $$

Como o denominador denotado por $P(tweet)$ aparece em ambas as equações, ele acaba sendo desconsiderado nos cálculos comparativos das probabilidades.

Por conseguinte, conclui-se que:

  $\quad \Rightarrow$ Se $P(R|tweet) > P(I|tweet)$, o tweet será classificado como **Relevante**.

  $\quad \Rightarrow$ Se $P(R|tweet) < P(I|tweet)$, o tweet será classificado como **Irrelevante**.

### Independência de palavras - O lado ingênuo do classificador:

As probabilidades $P(tweet|R)$ e $P(tweet|I)$ podem ser representadas considerando-se que haja uma independência das palavras em um dado tweet. Justamente por esse motivo, o classificador é considerado ingênuo, por desconsiderar a ordenação das palavras e analisá-las de forma independente, de modo que a probabilidade de uma palavra ocorrer na frase não influencia na probabilidade de outra ocorrer

$$ P(R|tweet) = \frac{P(palavra_1|R)P(palavra_2|R)...P(palavra_n|R) P(R)}{P(tweet)}$$

$$P(I|tweet)=\frac{P(palavra_1|I)P(palavra_2|I)...P(palavra_n|I) P(I)}{P(tweet)}$$

### Suavização de Laplace:

No caso de aparecer uma palavra que não esteja na base de dados, precisaremos recorrer à Suavização de Laplace, que inclui uma nova palavra nas palavras classificadas como relevantes ou irrelevantes. Para isso, deve-se observar quantas vezes a palavra analisada aparece na comparação em questão, seja relevante ou irrelavante, somar uma unidade no numerador evitando o 0, e somar a quantidade representadas pelas possíveis palavras no denominador, isto é a quantidade de palavras únicas pertencentes aos tweets relevantes ou irrelevantes.

Dessarte, ter-se-á as probabilidades $P(tweet|R)$ e $P(tweet|I)$ adequadas para receber as novas palavras.

$$P(palavra1|R) = \frac{F_{AR}+1}{P_{R}+P_p}$$

$$P(palavra1|I) = \frac{F_{AI}+1}{P_{I}+P_p}$$
 
Sendo: 

$ F_{AR}$: Frequência absoluta da palavra relevante 

$ F_{AI}$: Frequência absoluta da palavra irrelevante 
    
$P_{R}$: Todas as palavras pertencentes aos tweets relevantes
    
$P_{I}$: Todas as palavras pertencentes aos tweets irrelevantes

$P_p$: Todas as palavras possíveis na base de dados de treinamento. Em outrar palavras, esse é o tamanho do vocabulário.

In [14]:
# Tweets relevantes da planilha de Treinamento
train1 = train.loc[train.Classificação == 1]
# Tweets irrelevantes da planilha de Treinamento
train2 = train.loc[train.Classificação == 0]

In [15]:
# Series com as palavras da planilha Treinamento
palavras_treinamento = pd.Series(treinamento(train))
# Series com as palavras relevantes da planilha Treinamento
palavras_relevantes = pd.Series(treinamento(train1))
# Series com as palavras irrelevantes da planilha Treinamento
palavras_irrelevantes = pd.Series(treinamento(train2))
# Ocorrência das palavras da planilha Treinamento
planilha_treinamento = palavras_treinamento.value_counts()
tamanho_vocabulario = len(planilha_treinamento)

In [16]:
# Ocorrência das palavras relevantes da planilha Treinamento
relevantes = palavras_relevantes.value_counts()
# Ocorrência das palavras irrelevantes da planilha Treinamento
irrelevantes = palavras_irrelevantes.value_counts()

In [17]:
# Quantidade de palavras sem repetições da planilha Treinamento
total = len(relevantes) + len(irrelevantes)

In [18]:
# Freq. relativa das palavras relevantes da planilha Treinamento
relevantes_rel = palavras_relevantes.value_counts(True)
# Freq. relativa das palavras irrelevantes da planilha Treinamento
irrelevantes_rel = palavras_irrelevantes.value_counts(True)

In [19]:
total_relevantes = relevantes.sum()
total_irrelevantes = irrelevantes.sum()
total_palavras = total_relevantes + total_irrelevantes

In [20]:
# Tweets relevantes da planilha de Teste
test1 = test.loc[test.Classificação == 1]
# Tweets relevantes da planilha de Teste
test2 = test.loc[test.Classificação == 0]

In [21]:
# Aplica as funções anteriores e retorna uma lista de palavras do dataframe.Teste
def teste(dataframe):
    palavras = []
    for linha in dataframe.Teste:
        linha = separa_emoji(cleanup(linha))
        for palavra in limpa_marcacao(linha.split()):
            palavras.append(palavra)
    return palavras

In [22]:
# Aplica as funções anteriores e retorna uma lista de tweets (frases) do dataframe.Teste
def teste1(dataframe):
    frases = []
    for linha in dataframe.Teste:
        linha = limpa_marcacao1(separa_emoji(cleanup(linha)))
        frases.append(linha)
    return frases

In [23]:
# Probabilidades iniciais
probR = total_relevantes/total_palavras 
probI = total_irrelevantes/total_palavras

In [24]:
def classifica(dataframe):
    '''
    Função que, dada a planilha Teste, irá classificar, tweet a tweet, a probabilidade dele ser relevante ou não 
    '''
    algoritmo = []
    for frase in teste1(dataframe):
        probPalavraDadoR = 1
        probPalavraDadoI = 1
        for palavra in frase:
            if not palavra in relevantes.keys(): # Se a palavra não estiver na lista de relevantes, sua ocorrência será 0
                relevantes[palavra] = 0
            if not palavra in irrelevantes.keys(): # Se a palavra não estiver na lista de irrelevantes, sua ocorrência será 0
                irrelevantes[palavra] = 0

            probPalavraDadoR *= (relevantes[palavra] + 1)/(total_relevantes + tamanho_vocabulario)
            probPalavraDadoI *= (irrelevantes[palavra] + 1)/(total_irrelevantes + tamanho_vocabulario)
        
        probRDadoPalavra = probPalavraDadoR*probR
        probIDadoPalavra = probPalavraDadoI*probI
        
        if probRDadoPalavra > probIDadoPalavra:
            algoritmo.append(1)
        else:
            algoritmo.append(0)
    return algoritmo
# Freq. absoluta das palavras de uma tag com repetições
# Tamanho do vocabulário: quantidade de palavras considerando as duas tags, sem repetições

In [25]:
test.loc[:, 'Algoritmo'] = classifica(test)
test.head(10)

Teste  Classificação  Algoritmo
0                                   amo homem aranha              1          1
1  @drika_nozes na verdade esse é o plot de homem...              0          0
2  @vicgabil @cinepop falou que vai estrear na me...              0          1
3  @goodnerd23 os dois👍 só n sei como vou aguenta...              0          1
4                @cacocardassi homem aranha, óbvio!!              1          1
5  @racionalxbox @gamerspubbr só o homem aranha j...              1          1
6  sim eu sou viúvo de tobey maguire e andrew gar...              1          1
7  @caduaroso pinguim e falcone são mafiosos\nban...              0          0
8  toda vez que eu comento num post do dalau q eu...              0          0
9  jogo do homem aranha 2 e do wolverine anunciad...              1          1

___
## Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [26]:
matriz_de_comparacao = pd.crosstab(test["Classificação"] , test["Algoritmo"], normalize=True)*100
matriz_de_comparacao

Algoritmo         0     1
Classificação            
0              15.5  45.5
1               5.5  33.5

In [27]:
print("Acurácia: {0:.2f}%".format(matriz_de_comparacao[0][0] + matriz_de_comparacao[1][1]))

Acurácia: 49.00%


- O percentual de **verdadeiros positivos** do nosso classificador Naive-Bayes foi de **33,50%**
- O percentual de **verdadeiros negativos** do nosso classificador Naive-Bayes foi de **15,50%**
- O percentual de **falsos positivos** do nosso classificador Naive-Bayes foi de **45,50%**
- O percentual de **falsos negativos** do nosso classificador Naive-Bayes foi de **5,50%**

Tendo isso em vista, nosso classificador apresentou uma **acurácia** de **49,00%**

Para efeitos de visualização, usou-se a própria planilha de Treinamento para realizar-se comparação da classificação do algoritmo com a feita manualmente, para assegurar que não houve grandes problemas com o código.

In [28]:
def classifica(dataframe):
    '''
    Função que, dada a planilha Teste, irá classificar, tweet a tweet, a probabilidade dele ser relevante ou não 
    '''
    algoritmo = []
    for frase in treinamento1(dataframe):
        probPalavraDadoR = 1
        probPalavraDadoI = 1
        for palavra in frase:
            if not palavra in relevantes.keys(): # Se a palavra não estiver na lista de relevantes, sua ocorrência será 0
                relevantes[palavra] = 0
            if not palavra in irrelevantes.keys(): # Se a palavra não estiver na lista de irrelevantes, sua ocorrência será 0
                irrelevantes[palavra] = 0

            probPalavraDadoR *= (relevantes[palavra] + 1)/(total_relevantes + tamanho_vocabulario)
            probPalavraDadoI *= (irrelevantes[palavra] + 1)/(total_irrelevantes + tamanho_vocabulario)
        
        probRDadoPalavra = probPalavraDadoR*probR
        probIDadoPalavra = probPalavraDadoI*probI
        
        if probRDadoPalavra > probIDadoPalavra:
            algoritmo.append(1)
        else:
            algoritmo.append(0)
    return algoritmo

In [29]:
train.loc[:, 'Algoritmo'] = classifica(train)

In [30]:
matriz_de_comparacao = pd.crosstab(train["Classificação"] , train["Algoritmo"], normalize=True)*100
matriz_de_comparacao

Algoritmo              0          1
Classificação                      
0              38.795987   5.685619
1               0.000000  55.518395

In [31]:
print("Acurácia: {0:.2f}%".format(matriz_de_comparacao[0][0] + matriz_de_comparacao[1][1]))

Acurácia: 94.31%


Como a acurácia, neste testagem, apresentou uma alta porcentagem, de 94,31%, o algoritmo funcionou de acordo com o esperado, podendo-se concluir, portanto, que não houve alguma incoveniência notória na maneira como o código deste arquivo foi estruturado.

___
## Concluindo

Tendo-se em vista os percentuais obtidos com o nosso classificador Naive-Bayes acerca do produto ou tema escolhido, pode-se concluir que este apresentou um funcionamento razoável. Apesar de sua performance não ter sido uma das mais consideráveis, muitos foram os motivos pelos quais o classificador mostrou um percentual de acurácia quase que parcialmente efetivo, de 49,00%.
Entre eles, é preciso considerar-se a existência de frases sarcásticas ou que apresentam dupla negação, pois, como já comentado anteriormente, o classificador construído com os recursos utilizados é ingênuo, e isso se abrange a esses tipos de frases que podem ser bastante comuns no mundo humano, mas são extremamente complexas para um algoritmo compreender de maneira genuina. Outra questão a ser levantada é a presença do cada vez mais comentado viés tecnológico.

### O viés na tecnologia: um ponto simples, mas que merece atenção! 


Esse dito viés tecnológico é mais facilmente compreendido como um comportamento tendencioso que uma dada tecnologia pode apresentar, ou talvez sempre apresente, por um simples motivo: um determinado sistema que faz o uso de um algoritmo, seja um classificador como o nosso, ou uma tecnologia que usufrui de inteligência artificial, ou um sistema de reconhecimento facial, por exemplo, foi programado por ninguém mais, ninguém menos que um humano que, por sinal, possui determinadas crenças, percepções e uma visão de mundo únicas, de forma que, se esse sistema não for analisado detalhadamente, este pode apresentar um comportamento tendencioso que lhe foi passado através dos vieses inconcientes da pessoa por detrás dessa feitura. Um exemplo clássico e que vem chamando muito a atenção é o fato de determinadas tecnologias de reconhecimento facial implementadas por alguns países não reconhecerem facilmente pessoas de cor preta ou, no caso de algoritmos de detecção de perfis de risco, acusarem com muito mais incidência pessoas negras em comparação com brancas. 

Em se tratando do nosso caso, menos grave que os exemplificados acima, a propósito, como foi comentado que um dos critérios para que um tweet fosse classificado como relevante era ter mensagens acerca de HQs do universo Homem-Aranha, isso, ainda assim, é algo relativo, pois há uma diferença entre, por exemplo, citar-se apenas um HQ referente ao universo Homem-Aranha e em falar-se de uma maneira com muita propriedade acerca de fatos, personagens, tramas a respeito das obras ficcionais de Homem-Aranha.

### Por que  não podemos usar o próprio classificador para gerar mais amostras de treinamento?

Nosso algoritmo de Naive Bayes tenta classificar outros tweets com base no que ela aprendeu observando a base de treinamento. Isso condiciona o nosso classificador a classificar sempre baseado na planilha que ele tem acesso e, com isso, ele se torna tendencioso. Para que fosse possível gerar mais amostras seria necessário ir atualizando a base de treinamento periodicamente para que o classificador aumente sua base de dados e fique mais preparado para classificar novos tweets de forma mais eficiente.

### Plano de Expansão - O Naive Bayes em diferentes contextos

Como o classificador Naive Bayes presente neste arquivo foi testado em tweets de tamanho similar aos quais ele foi treinado, a partir da base de dados de treinamento, pudemos perceber que ele funcionou de forma razoável. Se considerássemos que ele atuasse com uma base de dados de teste mais sucinta, com cada tweet contendo apenas palavras-chave, é coerente dizer que a eficácia do classificador aumentaria, tendo em vista que este atua com base principalmente nessas palavras-chave mais comuns observadas após a limpeza dos tweets. Assim, é remetida uma situação muito recorrente no cenário atual, no qual as pesquisas, em seu sentido mais amplo, que são realizadas pelas diferentes faixas etárias, são traçadas de forma similar, a partir dessas palavras-chave. Dessa forma, o Naive-Bayes seria de grande utilidade para filtrar o que é mais pesquisado, que passa a ser classificado como relevante, em princípio, para fazer melhorias nas sugestões, seja de sites, vídeos, ou se filmes e séries, considerando o uso de plataforma streamings. No entanto, é claro: para que isso aconteça, serão necessárias variadas iterações prévias e, dessa forma, ganhar-se-á cada vez mais notoriedade por impresas interessadas.

Outra aplicação do classificador seria na filtragem e no reconhecimento da opinião do público geral a respeito do andamento das obras do universo Homem-Aranha, para que se torne facilitada a captura de informações, possobilitando que haja um embasamento maior e mais rápido do que está interessante nas obras e deve continuar ocorrendo ou ainda ser melhorada, do que já não está apresentando mais tanta notoriedade pelo público.

___
## Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

No código abaixo, tentou-se realizar uma operação para verificar a qualidade do classificador, realizando-se, para isso, novas separações entre a base de Treinamento e Teste. Para isso, foi consultada a documentação da biblioteca sklearn, mas a sua implementação não foi totalmente concluída.

In [32]:
def limpeza(linha):
    f = ','.join(limpa_marcacao1(separa_emoji(cleanup(linha)))).replace(',',' ')
    return f

# Unindo as tabelas em uma
test_novo = pd.read_excel(filename, sheet_name = 'Teste')
test_novo = test_novo.rename(columns={'Teste':'Total'})

train_novo = pd.read_excel(filename, sheet_name = 'Treinamento')
train_novo = train_novo.rename(columns={'Treinamento':'Total'})

# Juntando em um único dataframe
data = pd.concat([train_novo,test_novo])

# Limpeza
data['Limpo'] = data['Total'].apply(limpeza)

Abaixo, fez-se apenas o mesmo procedimento para algumas variáveis também presentes no tópico sobre o classificador Naive Bayes, para facilitar a utilização da função classifica5.

In [33]:
# Tweets relevantes da planilha de Teste
test1 = test.loc[test.Classificação == 1]
# Tweets relevantes da planilha de Teste
test2 = test.loc[test.Classificação == 0]

# Aplica as funções anteriores e retorna uma lista de palavras do dataframe.Teste
def teste(dataframe):
    palavras = []
    for linha in dataframe.Teste:
        linha = separa_emoji(cleanup(linha))
        for palavra in limpa_marcacao(linha.split()):
            palavras.append(palavra)
    return palavras

# Aplica as funções anteriores e retorna uma lista de tweets (frases) do dataframe.Teste
def teste1(dataframe):
    frases = []
    for linha in dataframe.Teste:
        linha = limpa_marcacao1(separa_emoji(cleanup(linha)))
        frases.append(linha)
    return frases

# Series com as palavras da planilha Teste
palavras_test = pd.Series(teste(test))
# Series com as palavras relevantes da planilha Teste
palavras_relevantes_test = pd.Series(teste(test1))
# Series com as palavras irrelevantes da planilha Teste
palavras_irrelevantes_test = pd.Series(teste(test2))
# Ocorrência das palavras da planilha Teste
planilha_teste = palavras_test.value_counts()

# Ocorrência das palavras relevantes da planilha Teste
relevantes_test = palavras_relevantes_test.value_counts()
# Ocorrência das palavras irrelevantes da planilha Teste
irrelevantes_test = palavras_irrelevantes_test.value_counts()

total_test = len(relevantes_test) + len(irrelevantes_test)

probR = total_relevantes/total_palavras 
probI = total_irrelevantes/total_palavras

def classifica5(dataframe, relevantes, irrelevantes, total_relevantes, total_irrelevantes, tamanho_vocabulario):
    '''
    Função que, dada a planilha Teste, irá classificar, tweet a tweet, a probabilidade dele ser relevante ou não 
    '''
    algoritmo = []
    for frase in dataframe:
        probPalavraDadoR = 1
        probPalavraDadoI = 1
        for palavra in frase:
            if not palavra in relevantes.keys(): # Se a palavra não estiver na lista de relevantes, sua ocorrência será 0
                relevantes[palavra] = 0
            if not palavra in irrelevantes.keys(): # Se a palavra não estiver na lista de irrelevantes, sua ocorrência será 0
                irrelevantes[palavra] = 0

            probPalavraDadoR *= (relevantes[palavra] + 1)/(total_relevantes + tamanho_vocabulario)
            probPalavraDadoI *= (irrelevantes[palavra] + 1)/(total_irrelevantes + tamanho_vocabulario)
        
        probRDadoPalavra = probPalavraDadoR*probR
        probIDadoPalavra = probPalavraDadoI*probI
        
        if probRDadoPalavra > probIDadoPalavra:
            algoritmo.append(1)
        else:
            algoritmo.append(0)
    return algoritmo

# print(classifica(test))
# Freq. absoluta das palavras de uma tag com repetições
# Tamanho do vocabulário: quantidade de palavras considerando as duas tags, sem repetições

# Tweets relevantes da planilha de Treinamento
train1 = train.loc[train.Classificação == 1]
# Tweets irrelevantes da planilha de Treinamento
train2 = train.loc[train.Classificação == 0]

# Series com as palavras da planilha Treinamento
palavras_treinamento = pd.Series(treinamento(train))
# Series com as palavras relevantes da planilha Treinamento
palavras_relevantes = pd.Series(treinamento(train1))
# Series com as palavras irrelevantes da planilha Treinamento
palavras_irrelevantes = pd.Series(treinamento(train2))
# Ocorrência das palavras da planilha Treinamento
planilha_treinamento = palavras_treinamento.value_counts()
tamanho_vocabulario = len(planilha_treinamento)

# Ocorrência das palavras relevantes da planilha Treinamento
relevantes = palavras_relevantes.value_counts()
# Ocorrência das palavras irrelevantes da planilha Treinamento
irrelevantes = palavras_irrelevantes.value_counts()

# Quantidade de palavras sem repetições da planilha Treinamento
total = len(relevantes) + len(irrelevantes)

# Freq. relativa das palavras relevantes da planilha Treinamento
relevantes_rel = palavras_relevantes.value_counts(True)
# Freq. relativa das palavras irrelevantes da planilha Treinamento
irrelevantes_rel = palavras_irrelevantes.value_counts(True)

total_relevantes = relevantes.sum()
total_irrelevantes = irrelevantes.sum()
total_palavras = total_relevantes + total_irrelevantes

# Aplica as funções anteriores e retorna uma lista de palavras do dataframe.Treinamento
def treinamento5(dataframe):
    palavras = []
    for linha in dataframe.Limpo:
        for palavra in linha:
            palavras.append(palavra)
    return palavras

Por fim, abaixo, por algum motivo, provavelmente pequeno, não foi possível realizar as repetições das separações dos tweets. Portanto, não foi impresso, posteriormente, os novos valores de acurácia e o gráfico histograma.

In [34]:
# from sklearn.model_selection import train_test_split
# vdd_positivo = []
# vdd_negativo = [] 
# precisao      = []

# for i in range(0,10000):
#     X_train, X_test, y_train, y_test = train_test_split(data[['Limpo','Classificação']],data['Classificação'], test_size=0.4)
    
#     print(X_train)
#     train1 = X_train[X_train['Classificação'] == 1]
#     train2 = X_train[X_train['Classificação'] == 0]
    
#     palavras_treinamento = pd.Series(treinamento5(X_train))
    
#     palavras_relevantes = pd.Series(treinamento5(train1))
#     palavras_irrelevantes = pd.Series(treinamento5(train2))
#     planilha_treinamento = palavras_treinamento.value_counts()
#     tamanho_vocabulario = len(planilha_treinamento)
    
#     relevantes = palavras_relevantes.value_counts()
#     irrelevantes = palavras_irrelevantes.value_counts()
    
#     total = len(relevantes) + len(irrelevantes)
#     total # DUVIDA: len(palavras_treinamento)

#     relevantes_rel = palavras_relevantes.value_counts(True)
#     irrelevantes_rel = palavras_irrelevantes.value_counts(True)

#     total_relevantes = relevantes.sum()
#     total_irrelevantes = irrelevantes.sum()
#     total_palavras = total_relevantes + total_irrelevantes
    
#     probR = total_relevantes/total_palavras 
#     probI = total_irrelevantes/total_palavras

#     X_test.loc[:, 'Classificação'] = classifica5(X_test.Limpo, relevantes, irrelevantes, total_relevantes, total_irrelevantes, tamanho_vocabulario)

#     verdadeiro_positivo=X_test.loc[(X_test['Classificação']==1)&(X_test['Classificação']==1),:].shape[0]
#     verdadeiro_negativo=X_test.loc[(X_test['Classificação']==0)&(X_test['Classificação']==0),:].shape[0]
#     acuracia=(verdadeiro_positivo+verdadeiro_negativo)/X_test.shape[0]
    
#     P_ver_pos = (verdadeiro_positivo/X_test.shape[0])*100
#     P_ver_neg = (verdadeiro_negativo/X_test.shape[0])*100
#     acuracia *= 100
    
#     vdd_positivo.append(P_ver_pos)
#     vdd_negativo.append(P_ver_neg)
#     precisao.append(acuracia)    

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**